<a href="https://colab.research.google.com/github/R12942159/DeepLearning/blob/main/DLCV_p1_inferance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch


# Get cuda from GPU device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"

In [2]:
# delete
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
import sys


img_paths = sys.argv[1]
# img_paths = '/content/drive/MyDrive/DLCV/p1_data/val_50' # delete

# search file paths
img_paths_val = [os.path.join(img_paths, i) for i in os.listdir(img_paths) if i.endswith('.png')]

In [5]:
import torch
from PIL import Image
import numpy as np


class PretrainDataset(torch.utils.data.Dataset):
    def __init__(self, img_paths, transform):
        self.img_paths = img_paths
        self.transform = transform

    def __len__(self):
        """number of samples"""
        return len(self.img_paths)

    def __getitem__(self, idx):
        """read 1 sample"""
        # Read img
        path = self.img_paths[idx] # get img path
        img = Image.open(path).convert('RGB') # read img

        # transform img
        img = self.transform(img)

        # Read class index
        cls_idx = int((path.split('/')[-1]).split('_')[0])
        cls_idx = torch.tensor(cls_idx, dtype=torch.int64)

        return img, cls_idx

In [6]:
from torchvision.models import efficientnet_b4, EfficientNet_B4_Weights


# Preprocess Transform
transform =  EfficientNet_B4_Weights.DEFAULT.transforms()

# Build Dataset
val_ds = PretrainDataset(img_paths_val, transform)

In [7]:
# Build DataLoaders
BATCH_SIZE = 50
val_loader = torch.utils.data.DataLoader(val_ds, BATCH_SIZE, shuffle=False)

In [9]:
# Build Model and Replace classifier
from torchvision import models


model = models.efficientnet_b4(weights=EfficientNet_B4_Weights.DEFAULT)

last_layer = list(model.children())[-1]
weights = last_layer[1].weight
bias = last_layer[1].bias
new_linear_layer = torch.nn.Linear(in_features=weights.shape[1], out_features=50, bias=True)
new_linear_layer.weight.data = weights
new_linear_layer.bias.data = bias
last_layer[1] = new_linear_layer

model.load_state_dict(torch.load("/content/drive/MyDrive/DLCV/EFFICIENTNET_best_model.pth"))

<All keys matched successfully>

In [ ]:
model = model.to(device) # move model to GPU
model.eval() # Set model to evaluation mode

In [11]:
# Create temp tensor
y_true = torch.tensor([], dtype=torch.long).to(device)
y_pred_logits = torch.tensor([]).to(device)

In [12]:
from torch import nn


# deactivate autograd engine and reduce memory usage and speed up computations
with torch.no_grad():
    for batch_i, (x, y) in enumerate(val_loader):
        x, y = x.to(device), y.to(device)

        pred = model(x)
        # Concatenate Ground truth and prediction
        y_true = torch.cat((y_true, y), dim=0)
        y_pred_logits = torch.cat((y_pred_logits, pred), dim=0)

y_pred_cls = y_pred_logits.argmax(1) # get class idx with max prob (100, 3) -> (100, )
y_pred_probs = nn.Softmax(dim=1)(y_pred_logits) # convert logits to probability

In [13]:
# Move tensor to CPU and convert to np.ndarray
y_true = y_true.cpu().numpy()
y_pred_cls = y_pred_cls.cpu().numpy()

In [14]:
y_true, y_pred_cls

(array([27, 28,  8, ..., 35,  5, 25]), array([27, 28, 48, ..., 41,  4, 25]))

In [15]:
import csv
import sys

# out_csv = sys.argv[2]
out_csv = '/content/drive/MyDrive/DLCV'
try:
    os.makedirs(os.path.dirname(out_csv), exist_ok=True)
except:
    pass

with open('/content/drive/MyDrive/DLCV/out_csv.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(('filename', 'label'))
    for data in zip(y_true, y_pred_cls):
        writer.writerow(data)